In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC, SVR
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, mean_squared_error
import os

In [2]:
# Define los paths
classification_path = os.path.join(os.path.dirname(os.path.abspath('')), 'ejercicio_10', 'titanic_proc.csv')
regression_path = os.path.join(os.path.dirname(os.path.abspath('')), 'ejercicio_10', 'targets.csv')

In [3]:
# Carga los datasets
classification_data = pd.read_csv(classification_path)
regression_data = pd.read_csv(regression_path)

# Dataset de clasificación (Titanic)
X_class = classification_data.drop(columns='Survived')
y_class = classification_data['Survived']

# Dataset de regresión (Targets)
X_reg = regression_data.drop(columns='rotation')
y_reg = regression_data['rotation']

# Convierte variables categóricas a numéricas si es necesario
X_class = pd.get_dummies(X_class)
X_reg = pd.get_dummies(X_reg)

In [4]:
# Divide los datasets en entrenamiento y prueba
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(X_class, y_class, test_size=0.3, random_state=42)
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size=0.3, random_state=42)

In [5]:
# Pipelines
classification_pipelines = {
    'svm_class_pipeline': Pipeline([
        ('scaler', StandardScaler()),
        ('svm', SVC(kernel='rbf'))
    ]),
    'random_forest_class_pipeline': Pipeline([
        ('scaler', StandardScaler()),
        ('random_forest', RandomForestClassifier())
    ]),
    'gradient_boosting_class_pipeline': Pipeline([
        ('scaler', StandardScaler()),
        ('gbc', GradientBoostingClassifier())
    ]),
    'mlp_class_pipeline': Pipeline([
        ('scaler', StandardScaler()),
        ('mlp', MLPClassifier(max_iter=1000))
    ])
}

regression_pipelines = {
    'random_forest_reg_pipeline': Pipeline([
        ('scaler', StandardScaler()),
        ('random_forest', RandomForestRegressor())
    ]),
    'gradient_boosting_reg_pipeline': Pipeline([
        ('scaler', StandardScaler()),
        ('gbr', GradientBoostingRegressor())
    ]),
    'mlp_reg_pipeline': Pipeline([
        ('scaler', StandardScaler()),
        ('mlp', MLPRegressor(max_iter=1000))
    ]),
    'svr_reg_pipeline': Pipeline([
        ('scaler', StandardScaler()),
        ('svr', SVR())
    ])
}

pipelines = classification_pipelines | regression_pipelines

In [6]:
param_grids_classification = {
    'svm_class_pipeline': {
        'svm__C': [0.1, 1, 10],
        'svm__gamma': [0.01, 0.1, 1]
    },
    'random_forest_class_pipeline': {
        'random_forest__n_estimators': [100, 200],
        'random_forest__max_depth': [10, 20, None]
    },
    'gradient_boosting_class_pipeline': {
        'gbc__n_estimators': [100, 200],
        'gbc__learning_rate': [0.01, 0.1, 0.2]
    },
    'mlp_class_pipeline': {
        'mlp__hidden_layer_sizes': [(50,), (100,)],
        'mlp__alpha': [0.0001, 0.001]
    }
}

param_grids_regression = {
    'svr_reg_pipeline': {
        'svr__C': [0.1, 1, 10, 100],
        'svr__gamma': ['scale', 'auto', 0.01, 0.1, 1]
    },
    'random_forest_reg_pipeline': {
        'random_forest__n_estimators': [100, 200],
        'random_forest__max_depth': [10, 20, None]
    },
    'gradient_boosting_reg_pipeline': {
        'gbr__n_estimators': [100, 200],
        'gbr__learning_rate': [0.01, 0.1, 0.2]
    },
    'mlp_reg_pipeline': {
        'mlp__hidden_layer_sizes': [(50,), (100,)],
        'mlp__alpha': [0.0001, 0.001]
    }
}

param_grids = param_grids_classification | param_grids_regression

In [7]:
# Ejecuta GridSearchCV para clasificación y regresión
results = {}
for name, pipeline in pipelines.items():
    param_grid = param_grids.get(name, None)
    if param_grid:
        grid_search = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1)
        
        if 'class' in name:
            grid_search.fit(X_train_class, y_train_class)
            y_pred = grid_search.best_estimator_.predict(X_test_class)
            accuracy = accuracy_score(y_test_class, y_pred)
            results[name] = {'best_params': grid_search.best_params_, 'accuracy': accuracy}
            print(f"Accuracy ({name}):", accuracy)
            print(classification_report(y_test_class, y_pred))
        else:
            grid_search.fit(X_train_reg, y_train_reg)
            y_pred = grid_search.best_estimator_.predict(X_test_reg)
            mse = mean_squared_error(y_test_reg, y_pred)
            results[name] = {'best_params': grid_search.best_params_, 'mse': mse}
            print(f"Mean Squared Error ({name}):", mse)

Accuracy (svm_class_pipeline): 0.8059701492537313
              precision    recall  f1-score   support

           0       0.79      0.90      0.85       157
           1       0.83      0.67      0.74       111

    accuracy                           0.81       268
   macro avg       0.81      0.79      0.79       268
weighted avg       0.81      0.81      0.80       268

Accuracy (random_forest_class_pipeline): 0.8134328358208955
              precision    recall  f1-score   support

           0       0.80      0.90      0.85       157
           1       0.84      0.68      0.75       111

    accuracy                           0.81       268
   macro avg       0.82      0.79      0.80       268
weighted avg       0.82      0.81      0.81       268

Accuracy (gradient_boosting_class_pipeline): 0.8059701492537313
              precision    recall  f1-score   support

           0       0.79      0.92      0.85       157
           1       0.85      0.65      0.73       111

    accu

C:\Users\Carli\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
2 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Carli\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Carli\anaconda3\Lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Carli\anaconda3\Lib\site-packages\sklearn\pipeline.py", line 416, in fit
    Xt = self._fit(X, y, **fit_

Mean Squared Error (random_forest_reg_pipeline): 10886.684719744206
Mean Squared Error (gradient_boosting_reg_pipeline): 10861.156159310547
Mean Squared Error (mlp_reg_pipeline): 83924.50833558383
Mean Squared Error (svr_reg_pipeline): 10879.32558455637


In [8]:
# Actualiza el diccionario general de pipelines con los mejores modelos
for name, result in results.items():
    pipelines[name] = result['best_params']

In [10]:
results

{'svm_class_pipeline': {'best_params': {'svm__C': 1, 'svm__gamma': 0.1},
  'accuracy': 0.8059701492537313},
 'random_forest_class_pipeline': {'best_params': {'random_forest__max_depth': 20,
   'random_forest__n_estimators': 200},
  'accuracy': 0.8134328358208955},
 'gradient_boosting_class_pipeline': {'best_params': {'gbc__learning_rate': 0.01,
   'gbc__n_estimators': 200},
  'accuracy': 0.8059701492537313},
 'mlp_class_pipeline': {'best_params': {'mlp__alpha': 0.001,
   'mlp__hidden_layer_sizes': (50,)},
  'accuracy': 0.8097014925373134},
 'random_forest_reg_pipeline': {'best_params': {'random_forest__max_depth': 10,
   'random_forest__n_estimators': 200},
  'mse': 10886.684719744206},
 'gradient_boosting_reg_pipeline': {'best_params': {'gbr__learning_rate': 0.01,
   'gbr__n_estimators': 100},
  'mse': 10861.156159310547},
 'mlp_reg_pipeline': {'best_params': {'mlp__alpha': 0.001,
   'mlp__hidden_layer_sizes': (100,)},
  'mse': 83924.50833558383},
 'svr_reg_pipeline': {'best_params': 